In [1]:
import pynwb
from allensdk.brain_observatory.ecephys.dynamic_gating_ecephys_session import DynamicGatingEcephysSession
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

import xarray as xr
import sys

# Add the path you want to include
sys.path.append('aperiodic_shield/code')

from shield_utils import find_animals, get_lfp_dict, downsample, align_lfp

import json

data_path = '/oscar/data/sjones/kduecker/shield_data/'

meta_path = 'aperiodic_shield/externals/SHIELD_Dynamic_Gating_Analysis'


In [2]:
# find mice of interest

mice = find_animals(meta_path)


In [3]:
with open('aperiodic_shield/session_info.json', 'r') as file:
    session_info = json.load(file)

In [4]:
subj = session_info['sessions_all_areas'][0][0:6]

In [5]:
# get good subjects and good sessions from Christian's .json file
ses_files = os.listdir(os.path.join(data_path,f'sub-{subj}'))

# get lfp files and spike files
lfp_files = list(filter(lambda s: 'None' in s, ses_files))
session = session_info['sessions_all_areas'][0][7:-1]
spike_files = list(filter(lambda s: session in s, ses_files))


In [9]:
down_srate = 500
roi = ['LGd', 'VISp', 'VISl']
toi = [0, 2]
layer_lfp = get_lfp_dict(subj, data_path, lfp_files, spike_files[0], toi, down_srate, roi)

/oscar/home/kduecker/aperiodic_shield/code/shield_utils.py:54: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'time' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates in the future (i.e., one coordinate for each multi-index level + one dimension coordinate). If you want to keep this behavior, you need to first wrap it explicitly using `mindex_coords = xarray.Coordinates.from_pandas_multiindex(mindex_obj, 'dim')` and pass it as coordinates, e.g., `xarray.Dataset(coords=mindex_coords)`, `dataset.assign_coords(mindex_coords)` or `dataarray.assign_coords(mindex_coords)`.
  ds = ds.assign(time=inds).unstack('time')
/oscar/home/kduecker/aperiodic_shield/code/shield_utils.py:54: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'time' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates in the future (i.e., one coordinate for each multi-inde